In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import nltk
import sklearn

In [2]:
df = pd.read_csv('data/dataset.csv')

In [3]:
df = df.dropna().reset_index(drop=True)

In [4]:
df['Mean Proportion'] = (df['Dow Jones Proportion'] + df['Nasdaq Proportion'] + df['S&P 500 Proportion'])/3

In [5]:
df['Body'] = df['Body'].apply(lambda body: body + ' ').str.replace('\d+', '')

In [9]:
df

,Date,Title,Body,Nasdaq Value,Nasdaq Delta,Nasdaq Proportion,Dow Jones Value,Dow Jones Delta,Dow Jones Proportion,S&P 500 Value,S&P 500 Delta,S&P 500 Proportion,Mean
0,2017-04-06,Notice Regarding the Continuation of the Natio...,NOTICE- - - - - - -CONTINUATION OF THE NATIONA...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
1,2017-04-06,Message to the Congress Regarding the Continua...,TO THE CONGRESS OF THE UNITED STATES:Section(d...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
2,2017-04-06,1600 Daily: Everything White House for 4/6/17,"Summary:Get news, events and updates from the ...",5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
3,2017-04-06,Vice President Mike Pence to Travel to the Rep...,"WASHINGTON, DC - Vice President Mike Pence wil...",5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
4,2017-04-06,White House History: White House Easter Egg Roll,Continuing the timeless tradition of the White...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
5,2017-04-06,Remarks by President Trump and Vice President ...,East Room: A.M. EDTTHE VICE PRESIDENT: To all ...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
6,2017-04-06,"Photo of the Day: April 6, 2017",President Donald Trump and King Abdullah II of...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
7,2017-04-05,Readout of President Donald J. Trump’s Call wi...,President Donald J. Trump spoke today with Pri...,5864.479980,14.470215,0.002467,20648.150391,14.798828,0.000717,2352.949951,4.540039,0.001930,11.269694
8,2017-04-05,Statement from the Press Secretary on Gold Sta...,"On this day in , nearing the end of World War ...",5864.479980,14.470215,0.002467,20648.150391,14.798828,0.000717,2352.949951,4.540039,0.001930,11.269694
9,2017-04-05,Memorandum: Implementing Executive Order 13771...,M--MEMORANDUM FOR:REGULATORY POLICY OFFICERS A...,5864.479980,14.470215,0.002467,20648.150391,14.798828,0.000717,2352.949951,4.540039,0.001930,11.269694


In [10]:
df.groupby('Date')[['Dow Jones Delta', 'Nasdaq Delta', 'S&P 500 Delta']].mean()

,Dow Jones Delta,Nasdaq Delta,S&P 500 Delta
Date,,,
2017-02-07,-35.949218,8.229980,1.589844
2017-02-08,118.060547,32.729981,13.200195
2017-02-09,96.968750,18.949707,8.229981
2017-02-10,142.791015,29.830078,12.149902
2017-02-13,92.250000,18.609863,9.330078
2017-02-14,107.449219,36.870117,11.669922
2017-02-15,7.910156,-4.540039,-2.030029
2017-02-16,4.281250,23.680176,3.939941
2017-02-17,118.949219,27.370117,14.219971


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

n_samples = 20000
n_topics = 4
n_top_words = 25

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [12]:
# Vectorize our text using CountVectorizer
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.80, min_df=50,
                                max_features=None,
                                stop_words='english'
                                )

tf = tf_vectorizer.fit_transform(df['Body'])

Extracting tf features for LDA...


In [13]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_topics=%d..."
      % (n_samples, n_topics))

#define the lda function, with desired options
#Check the documentation, linked above, to look through the options
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=20,
                                learning_method='online',
                                learning_offset=80.,
                                total_samples=n_samples,
                                random_state=0)
#fit the model
lda.fit(tf)

Fitting LDA models with tf features, n_samples=20000 and n_topics=4...


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=80.0,
             max_doc_update_iter=100, max_iter=20, mean_change_tol=0.001,
             n_jobs=1, n_topics=4, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=20000, verbose=0)

In [14]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:

Topic #0:
going thank great people applause know want just good right really like country jobs mr american lot think im laughter time say job said years

Topic #1:
united states trump american country america security people minister prime vice world leaders economic today countries women work national jobs great americans donald years trade

Topic #2:
think mr going people just said know house question im administration want right make dont white look theres thats did sure say does lot time

Topic #3:
federal law order house trump executive department agency states state white mr secretary general court agencies rule government regulatory act office administration director attorney regulations



In [14]:
topic_dist = lda.transform(tf)
topic_dist_df = pd.DataFrame(topic_dist)
df_w_topics = topic_dist_df.join(df)
df_w_topics

,0,1,2,3,Date,Title,Body,Nasdaq Value,Nasdaq Delta,Nasdaq Proportion,Dow Jones Value,Dow Jones Delta,Dow Jones Proportion,S&P 500 Value,S&P 500 Delta,S&P 500 Proportion,Mean
0,0.274324,0.717448,0.004049,0.004178,2017-04-06,Notice Regarding the Continuation of the Natio...,NOTICE- - - - - - -CONTINUATION OF THE NATIONA...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
1,0.221069,0.763802,0.007477,0.007651,2017-04-06,Message to the Congress Regarding the Continua...,TO THE CONGRESS OF THE UNITED STATES:Section(d...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
2,0.051654,0.844772,0.051348,0.052226,2017-04-06,1600 Daily: Everything White House for 4/6/17,"Summary:Get news, events and updates from the ...",5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
3,0.002288,0.993145,0.002281,0.002286,2017-04-06,Vice President Mike Pence to Travel to the Rep...,"WASHINGTON, DC - Vice President Mike Pence wil...",5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
4,0.021665,0.935372,0.021194,0.021768,2017-04-06,White House History: White House Easter Egg Roll,Continuing the timeless tradition of the White...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
5,0.000987,0.156946,0.841061,0.001006,2017-04-06,Remarks by President Trump and Vice President ...,East Room: A.M. EDTTHE VICE PRESIDENT: To all ...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
6,0.021204,0.879085,0.022138,0.077573,2017-04-06,"Photo of the Day: April 6, 2017",President Donald Trump and King Abdullah II of...,5878.950195,-1.140136,-0.000194,20662.949219,-6.849610,-0.000331,2357.489990,-1.949951,-0.000827,-3.313232
7,0.008148,0.975173,0.008195,0.008484,2017-04-05,Readout of President Donald J. Trump’s Call wi...,President Donald J. Trump spoke today with Pri...,5864.479980,14.470215,0.002467,20648.150391,14.798828,0.000717,2352.949951,4.540039,0.001930,11.269694
8,0.009405,0.552839,0.428205,0.009551,2017-04-05,Statement from the Press Secretary on Gold Sta...,"On this day in , nearing the end of World War ...",5864.479980,14.470215,0.002467,20648.150391,14.798828,0.000717,2352.949951,4.540039,0.001930,11.269694
9,0.999443,0.000186,0.000185,0.000187,2017-04-05,Memorandum: Implementing Executive Order 13771...,M--MEMORANDUM FOR:REGULATORY POLICY OFFICERS A...,5864.479980,14.470215,0.002467,20648.150391,14.798828,0.000717,2352.949951,4.540039,0.001930,11.269694


In [17]:
sorted_topic_1 = df_w_topics[['Title', 'Date', 'Mean', 3]].sort_values(by=[3], ascending=False)
sorted_topic_1['Mean'].mean()

10.516422976241946

In [ ]:
# top articles with >= correlation to topic 0
topic0_top = sorted_topic_0[sorted_topic_0[0] >= 0.8]

# top articles with >= correlation to topic 1
topic1_top = sorted_topic_1[sorted_topic_1[1] >= 0.8]

# top articles with >= correlation to topic 2
topic2_top = sorted_topic_2[sorted_topic_2[2] >= 0.8]

# top articles with >= correlation to topic 3
topic3_top = sorted_topic_3[sorted_topic_3[3] >= 0.8]

In [ ]:
topics = [topic0_top, topic1_top, topic2_top, topic3_top]

for t1, t2 in list(combinations(np.arange(len(topics)), 2)):
    print("Pvalue for mean proportion betwen topic #{} ({} posts) and topic #{} ({} posts)"\
          .format(t1, len(topics[t1]), t2, len(topics[t2])))
    print(ttest_ind(topics[t1]['Mean Proportion'], topics[t2]['Mean Proportion'], equal_var = False)[1])

In [ ]:
vectorizer = CountVectorizer(encoding='utf-8', stop_words='english')
dtm_upward = vectorizer.fit_transform(df_upward_grouped['Body'].sum())

In [ ]:
dtm_df = pd.DataFrame(vectorizer.fit_transform(df_downward_grouped['Body'].sum()).toarray(), columns=vectorizer.get_feature_names())

In [ ]:
print(dtm_df.sum().sort_values(ascending=False))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvec = TfidfVectorizer(stop_words = 'english')

#create the dtm, but with cells weigthed by the tf-idf score.
dtm_tfidf_df = pd.DataFrame(tfidfvec.fit_transform(df_downward_grouped['Body'].sum()).toarray(), columns=tfidfvec.get_feature_names())

#view results
dtm_tfidf_df

In [ ]:
print(dtm_tfidf_df.max().sort_values(ascending=False)[:20])